# Change to current dir


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
path="/content/drive/MyDrive/ColabNotebooks/_/"
os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/ColabNotebooks/_'

In [ ]:
!ls

6_1_EXE_Kaggle_Leaf_Challenge_LinManxi.ipynb  newDataset.csv	train.csv
example.csv				      numClassData.csv	tut1-model.pt
imageClassifier.ipynb			      test
model.ckpt				      train


In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

from IPython.display import clear_output
from skimage.io import imread
from skimage.transform import resize
                                     

# Read all train data [name, label]

In [ ]:
import csv
from collections import defaultdict

label = defaultdict(list)
with open('train.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        row = ''.join(row)
        row = row.split(' ')
        if row[0].isdigit():
            label[row[1]].append(row[0])

In [ ]:
Connective, Immune, Cancer, Normal = [], [], [], []
Connective = np.array(label.get('Connective'))
Immune = np.array(label.get('Immune'))
Cancer = np.array(label.get('Cancer'))
Normal = np.array(label.get('Normal'))

# Extend Dataset(roate and mirror)

In [ ]:
def roate_img(data):
    tmp = data
    for idx in tmp:
         img = Image.open('train/train/'+idx+'.png')    
         # roate 90 degree
         '''
         img = img.transpose(Image.ROTATE_90)
         img.save('train/train/'+str(int(idx)+2190)+'.png')
         tmp = np.append(tmp, str(int(idx)+2190))
        '''
         # mirror
        img = img.transpose(Image.FLIP_LEFT_RIGHT)
        img.save('train/train/'+str(int(idx)+4380)+'.png')
        tmp = np.append(tmp, str(int(idx)+4380))
    
     return tmp


In [ ]:
Normal_r = roate_img(Normal)

In [ ]:
Normal_m = roate_img(Normal_r)

In [ ]:
print(len(Connective), len(Immune), len(Cancer), len(Normal_m))

In [ ]:
Connective_ =  np.random.choice(Connective, size=512)
Immune_=np.random.choice(Immune, size=512)
Cancer_ = np.random.choice(Cancer, size=512)
Normal_ = np.random.choice(Normal_m, size=512)

# Store csv file [path, label]

In [ ]:
with open('numClassData1.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(['Id','Type'])
    for co in Connective_:
        writer.writerow(['train/train/'+co+'.png', 0])
    for im in Immune_:
        writer.writerow(['train/train/'+im+'.png', 1])
    for ca in Cancer_:
        writer.writerow(['train/train/'+ca+'.png', 2])
    for no in Normal_:
        writer.writerow(['train/train/'+no+'.png', 3])

f.close()

# Load the data


In [ ]:
for file in os.listdir(path):
print(file)

example.csv
train.csv
test
train
6_1_EXE_Kaggle_Leaf_Challenge_LinManxi.ipynb
imageClassifier.ipynb


In [ ]:
import pandas as pd

data = pd.read_csv('numClassData1.csv')

In [ ]:
print(data.info())
print(data.tail(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2048 entries, 0 to 2047
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      2048 non-null   object
 1   Type    2048 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 32.1+ KB
None
                        Id  Type
2043  train/train/3430.png     3
2044  train/train/7761.png     3
2045   train/train/999.png     3
2046  train/train/7714.png     3
2047  train/train/8297.png     3


In [ ]:
data = data.to_numpy()

In [ ]:
print(data.shape)

(2048, 2)


# Split dataset 
## Train: 0.9 Test: 0.1

In [ ]:
from sklearn.model_selection import train_test_split

X, y = data[:, 0], data[:, 1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)#随机选择25%作为测试集，剩余作为训练集


In [ ]:
# /train/train +./+1.png
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
X_train.shape


(1843,)

# Dataset and Dataloader

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
    #transforms.Scale(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor()
])

def default_loader(path):
    img_pil =  Image.open(path)
    # img_pil = img_pil.resize((224,224))
    img_tensor = preprocess(img_pil)
    # img_tensor = img_pil.ToTensor()
    return img_tensor


class trainset(Dataset):
    def __init__(self, file_train, label_train, loader=default_loader):
       
        self.images = file_train
        self.target = label_train
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.images[index], self
        img = self.loader(fn)
        target = self.target[index]
        # target = torch.tensor(target)
        return img, target

    def __len__(self):
        return len(self.images)

In [ ]:
batch_size = 128

## Train, Test dataset and dataloader

In [ ]:
train_data  = trainset(X_train, y_train)
trainloader = DataLoader(train_data, batch_size,shuffle=True)


In [ ]:
test_data  = trainset(X_test, y_test)
testloader = DataLoader(test_data, batch_size,shuffle=True)

In [ ]:
train_data_all  = trainset(X, y)
trainloader_all = DataLoader(train_data_all, batch_size=1,shuffle=True)


In [ ]:
from PIL import Image

In [ ]:
print(type(trainloader))
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images), type(labels))
print(images.size(), labels.size())

<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([128, 3, 64, 64]) torch.Size([128])


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

In [ ]:
# Hyper parameters
num_epochs = 20
num_classes = 4
batch_size = 128
learning_rate = 0.0001

In [ ]:
# output w = (w-f+2p)/s +1
#output channel = filter number

# ResNet

In [ ]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [ ]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=4):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8,ceil_mode=False) 
        self.fc = nn.Linear(256, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample)) 
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Train (ResNet)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# model = CNN()
# print(model)
model = ResNet(ResidualBlock, [2, 2, 2, 2]).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:

def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
print(len(trainloader))

15


In [ ]:
total_step = len(trainloader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
   
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/20], Step [1/15] Loss: 0.4361
Epoch [1/20], Step [2/15] Loss: 0.3286
Epoch [1/20], Step [3/15] Loss: 0.3107
Epoch [1/20], Step [4/15] Loss: 0.2744
Epoch [1/20], Step [5/15] Loss: 0.3312
Epoch [1/20], Step [6/15] Loss: 0.3336
Epoch [1/20], Step [7/15] Loss: 0.2434
Epoch [1/20], Step [8/15] Loss: 0.2838
Epoch [1/20], Step [9/15] Loss: 0.2705
Epoch [1/20], Step [10/15] Loss: 0.2508
Epoch [1/20], Step [11/15] Loss: 0.2810
Epoch [1/20], Step [12/15] Loss: 0.1954
Epoch [1/20], Step [13/15] Loss: 0.3020
Epoch [1/20], Step [14/15] Loss: 0.2360
Epoch [1/20], Step [15/15] Loss: 0.1733
Epoch [2/20], Step [1/15] Loss: 0.2267
Epoch [2/20], Step [2/15] Loss: 0.2634
Epoch [2/20], Step [3/15] Loss: 0.1981
Epoch [2/20], Step [4/15] Loss: 0.2690
Epoch [2/20], Step [5/15] Loss: 0.3418
Epoch [2/20], Step [6/15] Loss: 0.2138
Epoch [2/20], Step [7/15] Loss: 0.2570
Epoch [2/20], Step [8/15] Loss: 0.2279
Epoch [2/20], Step [9/15] Loss: 0.1958
Epoch [2/20], Step [10/15] Loss: 0.2630
Epoch [2/20], Step

In [ ]:
# Train the model
total_step = len(trainloader_all)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader_all):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Streaming output truncated to the last 5000 lines.
Epoch [23/25], Step [1145/2048], Loss: 0.0000
Epoch [23/25], Step [1146/2048], Loss: 0.0000
Epoch [23/25], Step [1147/2048], Loss: 0.0000
Epoch [23/25], Step [1148/2048], Loss: 0.0000
Epoch [23/25], Step [1149/2048], Loss: 0.0000
Epoch [23/25], Step [1150/2048], Loss: 0.0000
Epoch [23/25], Step [1151/2048], Loss: 0.0000
Epoch [23/25], Step [1152/2048], Loss: 0.0000
Epoch [23/25], Step [1153/2048], Loss: 0.0000
Epoch [23/25], Step [1154/2048], Loss: 0.0000
Epoch [23/25], Step [1155/2048], Loss: 0.0000
Epoch [23/25], Step [1156/2048], Loss: 0.0000
Epoch [23/25], Step [1157/2048], Loss: 0.0000
Epoch [23/25], Step [1158/2048], Loss: 0.0000
Epoch [23/25], Step [1159/2048], Loss: 0.0000
Epoch [23/25], Step [1160/2048], Loss: 0.0000
Epoch [23/25], Step [1161/2048], Loss: 0.0000
Epoch [23/25], Step [1162/2048], Loss: 0.0000
Epoch [23/25], Step [1163/2048], Loss: 0.0000
Epoch [23/25], Step [1164/2048], Loss: 0.0000
Epoch [23/25], Step [1165/204

# Test


In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model4_resnet.ckpt')

Test Accuracy of the model on the test images: 89.7560975609756 %


In [ ]:
test_file_list = os.listdir('test/test/')
print(len(test_file_list))


400


# Result


In [ ]:
import csv

## Read test file, store into csv


In [ ]:
with open('validation2.csv', 'a') as f:

    writer = csv.writer(f)
    writer.writerow(['Id','Type'])
    for pic in test_file_list:
        writer.writerow(['test/test/'+pic, pic[:-4]])


In [ ]:
f.close()

In [ ]:
val_data = pd.read_csv('validation2.csv')


In [ ]:
print(val_data.info)

<bound method DataFrame.info of                       Id   Type
0    test/test/10055.png  10055
1    test/test/10005.png  10005
2    test/test/10028.png  10028
3    test/test/10010.png  10010
4    test/test/10008.png  10008
..                   ...    ...
395  test/test/10302.png  10302
396  test/test/10317.png  10317
397  test/test/10359.png  10359
398  test/test/10368.png  10368
399  test/test/10295.png  10295

[400 rows x 2 columns]>


In [ ]:
val_data = val_data.to_numpy()
val_X = val_data[:, 0]
val_y = val_data[:, 1]

In [ ]:
val_data.shape

(400, 2)

## Validation dataset and dataloader
 

In [ ]:

validation_data  = trainset(val_X, val_y)
valloader = DataLoader(validation_data, batch_size=1,shuffle=True)

## Predict

In [ ]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
predicts = []
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicts.append([predicted.tolist()[0], labels.tolist()[0]])


    



In [ ]:
predicts[0]


[0, 10002]

## Predict Result

In [ ]:
#Connective: 0, Immune: 1, Cancer: 2, Normal: 3
with open('result_resnet2.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(['Id','Type'])
    for row in predicts:
        if row[0]==0:
            writer.writerow([row[1], 'Connective'])
        elif row[0]==1:
            writer.writerow([row[1], 'Immune'])
        elif row[0]==2:
            writer.writerow([row[1], 'Cancer'])
        elif row[0]==3:
            writer.writerow([row[1], 'Normal'])

    # for i in range(len(val_X)):
    #     if predicts[i][0]==0:
    #         writer.writerow([val_X[i][10:-4], 'Connective'])
    #     elif predicts[i][0]==1:
    #         writer.writerow([val_X[i][10:-4], 'Immune'])
    #     elif predicts[i][0]==2:
    #         writer.writerow([val_X[i][10:-4], 'Cancer'])
    #     elif predicts[i][0]==3:
    #         writer.writerow([val_X[i][10:-4], 'Normal'])


        
f.close()
        